In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import radians, sin, cos, asin, sqrt, pi, atan, atan2, fabs

In [2]:
def calcular_distancia(lat1,long1,lat2,long2):
    lon1, lat1 = (radians(coord) for coord in (long1,lat1))
    lon2, lat2 = (radians(coord) for coord in (long2,lat2))
    dlat = (lat2 - lat1)
    dlon = (lon2 - lon1)
    a = (
        sin(dlat * 0.5)**2 +
        cos(lat1) * cos(lat2) * sin(dlon * 0.5)**2
    )
    
    radioTierra = 6371008.8
    return 2 * radioTierra * asin(sqrt(a)) / 1000

#distancia de Haversine
#https://gist.github.com/habibutsu/8bbcc202a915e965c6a6d4f561d0e482

In [5]:
def distancia_minima(fila, dataframe):
    lat, lon = fila
    distancias = []
    for i in range (len(dataframe)):
        d = calcular_distancia(lat,lon,dataframe.lat[i], dataframe.lon[i])
        distancias.append(d)
    return min(distancias)

In [6]:
def cantidad_a_distancia(fila, dist, dataframe):
    lat, lon = fila
    cantidad = 0
    for i in range (len(dataframe)):
        d = calcular_distancia(lat,lon,dataframe.lat[i], dataframe.lon[i])
        if d <= dist:
            cantidad += 1
    return cantidad

## Propiedades solo de Capital

In [ ]:
propiedades = pd.read_csv('properati.csv')
#propiedades = pd.read_csv('/home/agustin/Escritorio/escritorio/fiuba/Organizacion de datos/properati.csv')

In [ ]:
solo_capital = propiedades.loc[(propiedades.state_name == 'Capital Federal') & (propiedades.lat.notnull()) & (propiedades.lon.notnull()),:]
solo_capital = solo_capital.loc[((solo_capital.lat <= -34) & (solo_capital.lat >= -35)),:]
solo_capital = solo_capital.loc[((solo_capital.lon <= -58) & (solo_capital.lon >= -59)),:]

## Subtes

In [4]:
subtes = pd.read_csv('datos/estaciones-de-subte.csv', sep = ",")

In [ ]:
subtes.rename(columns = {'X': 'lon', 'Y': 'lat'}, inplace = True)

In [ ]:
solo_capital.loc[:,'distancia_subtes'] = solo_capital.loc[:,['lat','lon']].apply(lambda x: distancia_minima(x, subtes), axis = 1)

## Monumentos

In [6]:
monumentos = pd.read_csv('datos/monumentos.csv', sep = ",")

In [9]:
solo_capital.loc[:,'distancia_monumentos'] = solo_capital.loc[:,['lat','lon']].apply(lambda x: distancia_minima(x, monumentos), axis = 1)

## Ferrocarriles

In [8]:
ferrocarriles = pd.read_csv('datos/estaciones-de-ferrocarril.csv', sep = ";")

In [14]:
ferrocarriles.rename(columns = {'LAT': 'lat', 'LNG': 'lon' }, inplace = True)

In [ ]:
solo_capital.loc[:,'distancia_ferrocarril'] = solo_capital.loc[:,['lat','lon']].apply(lambda x: distancia_minima(x, ferrocarriles), axis = 1)

## Estadios

In [10]:
estadios = pd.read_csv('datos/estadios.csv', sep = ";")

In [14]:
estadios.rename(columns = {'LAT': 'lat', 'LONG': 'lon' }, inplace = True)

In [2]:
solo_capital.loc[:,'distancia_estadios'] = solo_capital.loc[:,['lat','lon']].apply(lambda x: distancia_minima(x, estadios), axis = 1)

## Colectivos

In [58]:
colectivos = pd.read_csv('datos/paradas-de-colectivo.csv', sep = ",")

In [ ]:
solo_capital.loc[:,'colectivos'] = solo_capital.loc[:,['lat','lon']].apply\
(lambda x: cantidad_a_distancia(x,0.5, colectivos), axis = 1)

## Servicios

In [5]:
bibliotecas = pd.read_csv('datos/servicios/bibliotecas.csv', sep = ";")
bibliotecas.rename(columns = {'BIBLIOTECA': 'Nombre', 'LAT': 'lat', 'LNG': 'lon' }, inplace = True)
bibliotecas.loc[:,'Servicio'] = 'Biblioteca'
bibliotecas = bibliotecas.loc[:,['Nombre', 'Servicio', 'lat', 'lon']]
bibliotecas.head(1)

,Nombre,Servicio,lat,lon
0,HILARIO ASCASUBI,Biblioteca,-34.623337,-58.492223


In [7]:
bomberos = pd.read_csv('datos/servicios/bomberos.csv', sep = ";")
bomberos.rename(columns = {'DCIA': 'Nombre', 'LAT': 'lat', 'LNG': 'lon' }, inplace = True)
bomberos.loc[:,'Servicio'] = 'Bomberos'
bomberos = bomberos.loc[:,['Nombre', 'Servicio', 'lat', 'lon']]
bomberos.head(1)

,Nombre,Servicio,lat,lon
0,SECCION PALERMO,Bomberos,-34.577526,-58.435809


In [34]:
comisarias = pd.read_csv('datos/servicios/comisarias.csv', sep = ",")
comisarias.loc[:,'Servicio'] = 'Comisaria'
comisarias = comisarias.loc[:,['Nombre', 'Servicio', 'lat', 'lon']]
comisarias.head(1)

,Nombre,Servicio,lat,lon
0,COMISARIA 52,Comisaria,-34.683121,-58.468944


In [38]:
hospitales = pd.read_csv('datos/servicios/hospitales.csv', sep = ";")
hospitales.rename(columns = {'NOMBRE': 'Nombre', 'LAT': 'lat', 'LNG': 'lon' }, inplace = True)
hospitales.loc[:,'Servicio'] = 'Hospital'
hospitales = hospitales.loc[:,['Nombre', 'Servicio', 'lat', 'lon']]
hospitales.head(1)

,Nombre,Servicio,lat,lon
0,HOSPITAL GENERAL DE NIÑOS PEDRO DE ELIZALDE,Hospital,-34.628847,-58.377551


In [39]:
universidades = pd.read_csv('datos/servicios/universidades.csv', sep = ";")
universidades.rename(columns = {'UNIVERSIDA': 'Nombre', 'LAT': 'lat', 'LNG': 'lon' }, inplace = True)
universidades.loc[:,'Servicio'] = 'Universidad'
universidades = universidades.loc[:,['Nombre', 'Servicio', 'lat', 'lon']]
universidades.head(1)

,Nombre,Servicio,lat,lon
0,FLACSO (FACULTAD LATINOAMERICANA DE CIENCIAS S...,Universidad,-34.602676,-58.39544


In [49]:
clinicas = pd.read_csv('datos/servicios/clinicas.csv', sep = ",")
clinicas.rename(columns = {'NOMBRE': 'Nombre', 'LAT': 'lat', 'LNG': 'lon' }, inplace = True)
clinicas.loc[:,'Servicio'] = 'Clinica'
clinicas = clinicas.loc[:,['Nombre', 'Servicio', 'lat', 'lon']]
clinicas.head(1)

,Nombre,Servicio,lat,lon
0,ALPI - INSTITUTO DE REHABILITACION MARCELO FIT...,Clinica,-34.59163,-58.417333


In [51]:
centros_medicos = pd.read_csv('datos/servicios/centros-medicos.csv', sep = ";")
centros_medicos.rename(columns = {'NOMBRE': 'Nombre', 'Y': 'lat', 'X': 'lon' }, inplace = True)
centros_medicos.loc[:,'Servicio'] = 'Centro Medico'
centros_medicos = centros_medicos.loc[:,['Nombre', 'Servicio', 'lat', 'lon']]
centros_medicos.head(1)

,Nombre,Servicio,lat,lon
0,CMB N° 2,Centro Medico,-34.622156,-58.482987


In [57]:
servicios = pd.concat([bibliotecas,bomberos,comisarias,hospitales,universidades,clinicas,centros_medicos], ignore_index=True)
servicios = servicios.loc[(servicios.lat.notnull()) & (servicios.lon.notnull()),:]
servicios.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 641 entries, 0 to 641
Data columns (total 4 columns):
Nombre      641 non-null object
Servicio    641 non-null object
lat         641 non-null float64
lon         641 non-null float64
dtypes: float64(2), object(2)
memory usage: 25.0+ KB


In [ ]:
solo_capital.loc[:,'servicios'] = solo_capital.loc[:,['lat','lon']].apply\
(lambda x: cantidad_a_distancia(x,0.8, servicios), axis = 1)

# funcion auxiliar

In [45]:
def abc (fila):
    a = fila.split(',')
    a = '.'.join(a)
    return float(a)
